 refer https://www.kaggle.com/waxbabi/light-gbm-benchmark-0-3692/code
#implement basic content-based prediction

In [3]:
"""
products_seq = order_products_prior.groupby("order_id").apply(lambda order: order['product_id'].tolist()).values
sentences = [[str(p) for p in ps] for ps in products_seq]
w2v = gensim.models.Word2Vec(sentences, size=5, window=5, min_count=2, workers=4)
product_w2v_raw = []
for k in w2v.wv.vocab.keys():
    l = w2v.wv[k].tolist()
    l.append(int(k))
    product_w2v_raw.append(l)
product_w2v = pd.DataFrame(product_w2v_raw, columns=['w2v_0', 'w2v_1', 'w2v_2', 'w2v_3', 'w2v_4', 'product_id']).set_index('product_id')
"""
"""
pnames = [re.sub(r'[^.0-9a-zA-Z]', ' ', line).lower().split() for line in products.product_name.astype(str).values]
w2v = gensim.models.Word2Vec(pnames, size=10, window=5, workers=4, min_count=1)
pname_vector = np.array([w2v.wv[pname].mean(axis=0) for pname in pnames])
product_w2v = pd.DataFrame(pname_vector).add_prefix('w2v_')
product_w2v['product_id'] = products['product_id']
product_w2v.set_index('product_id', inplace=True)
"""
"""
temp = pd.pivot_table(df[['product_id', 'order_dow']], index=['product_id'], columns=['order_dow'], aggfunc=len, fill_value=0)
product_dow = temp.div(temp.sum(axis=1), axis=0).add_prefix('pdow_').reset_index().set_index('product_id')
"""
"""
temp = pd.pivot_table(df[['product_id', 'order_hour_of_day']], index=['product_id'], columns=['order_hour_of_day'], aggfunc=len, fill_value=0)
product_hod = temp.div(temp.sum(axis=1), axis=0).add_prefix('phod_').reset_index().set_index('product_id')
"""

In [10]:
"""
from collections import defaultdict

def trend(df):
    user_max_order_number = defaultdict(int)
    user_product_history = defaultdict(list)
    for i, (u, p, n) in enumerate(zip(df['user_id'], df['product_id'], df['order_number'])):
        user_max_order_number[u] = max(user_max_order_number[u], n)
        user_product_history[(u, p)].append(n)
    temp = []
    for cnt, (k, v) in enumerate(user_product_history.iteritems()):
        user_id = k[0]
        product_id = k[1]
        max_order_number = user_max_order_number[user_id]
        r = [0, 0, 0, 0, 0]
        for vv in v:
            if max_order_number - vv <= 0: r[0] += 1
            if max_order_number - vv <= 1: r[1] += 1
            if max_order_number - vv <= 2: r[2] += 1
            if max_order_number - vv <= 5: r[3] += 1
            if max_order_number - vv <= 10: r[4] += 1
        r.append(user_id)
        r.append(product_id)
        temp.append(r)
    return pd.DataFrame(temp).rename(columns={5: 'user_id', 6: 'product_id'}).set_index(['product_id', 'user_id']).add_prefix('putrends_')
"""

In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
import gensim
import re

In [2]:
orders = pd.read_csv('../csv/orders.csv')

In [2]:
products = pd.read_csv('../csv/products.csv')

In [3]:
order_products_prior = pd.read_csv('../csv/order_products__prior.csv')

In [4]:
order_products_train = pd.read_csv('../csv/order_products__train.csv')

In [5]:
product_detail = order_products_prior.set_index('product_id').join(products.set_index('product_id')).reset_index()

In [6]:
aisle = product_detail.groupby('aisle_id').agg({'reordered': ['sum', 'count']})
aisle[('reordered', 'ratio')] = aisle[('reordered', 'sum')].div(aisle[('reordered', 'count')])
aisle = aisle.add_prefix('aisle_').reset_index().set_index('aisle_id')

In [7]:
department = product_detail.groupby('department_id').agg({'reordered': ['sum', 'count']})
department[('reordered', 'ratio')] = department[('reordered', 'sum')].div(department[('reordered', 'count')])
department = department.add_prefix('department_').reset_index().set_index('department_id')

In [9]:
df = order_products_prior.set_index('order_id').join(orders.set_index('order_id'))

In [10]:
product_user = df.groupby(['product_id', 'user_id']).agg({'reordered': ['sum', 'count'], 
'order_number': ['mean', 'min', 'max'], 'days_since_prior_order': ['mean'], 'add_to_cart_order': ['mean']})\
.reset_index().set_index(['product_id', 'user_id'])
product_user[('reordered', 'ratio')] = product_user[('reordered', 'sum')].div(product_user[('reordered', 'count')])
product_user = product_user.add_prefix('pu')

In [11]:
pu = pd.DataFrame(df.groupby(['product_id', 'user_id']).size().rename('count')).reset_index()

In [12]:
truncted_product_user = pu.copy()
truncted_product_user.loc[truncted_product_user['count'] > 2, 'count'] = 2

In [13]:
product_level = pd.pivot_table(truncted_product_user, index=['product_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('product_id')
product_level[('user_id', 'ratio')] = product_level[('user_id', 2)].div(product_level[('user_id', 1)])
product_level = product_level.add_prefix('plevel_')

In [14]:
user_level = pd.pivot_table(truncted_product_user, index=['user_id'], columns=['count'], aggfunc=len, fill_value=0)\
.reset_index().set_index('user_id').add_prefix('ulevel_')

In [15]:
product_v = df.reset_index().groupby('product_id').agg({'order_id': ['nunique'], 'user_id': ['nunique'], 
'reordered': ['sum', 'count'], 'add_to_cart_order': ['mean'], 'days_since_prior_order': ['mean']})\
.reset_index().set_index('product_id')
product_v[('reordered', 'ratio')] = product_v[('reordered', 'sum')].div(product_v[('reordered', 'count')])
product_v = product_v.add_prefix('p_')

In [16]:
user_v = df.reset_index().groupby('user_id').agg({'days_since_prior_order': ['mean', 'sum'], 'order_id': ['nunique'], 
'product_id': ['nunique'], 'reordered': ['sum', 'count']})\
.reset_index().set_index('user_id')
user_v[('reordered', 'ratio')] = user_v[('reordered', 'sum')].div(user_v[('reordered', 'count')])
user_v = user_v.add_prefix('u_')

In [17]:
user_order = df.groupby(['user_id', 'order_number']).agg({'add_to_cart_order': 'max'})\
.reset_index().groupby(['user_id']).agg({'add_to_cart_order': ['max', 'min', 'mean']}).add_prefix('uo_')

In [19]:
product_embeddings = pd.read_csv('product_embeddings.csv')
embedings = list(str(i) for i in range(32))
product_embeddings = product_embeddings[embedings + ['product_id']].set_index('product_id').add_prefix('pe_')

In [21]:
#product_user
#product_v, product_level, product_w2v, aisle, department, product_dow, product_hod
#user_level, user_v
#label

In [20]:
product_all = products.set_index('product_id').join(product_v).join(product_level).join(product_embeddings)\
.reset_index().set_index('aisle_id').join(aisle)\
.reset_index().set_index('department_id').join(department)\
.reset_index().set_index('product_id')\
.drop(['product_name', 'aisle_id', 'department_id'], axis=1)
product_all['p_avg_reordered_times'] = product_all[('p_reordered', 'p_sum')].div(product_all[('plevel_user_id', 'plevel_2')])
#.join(product_dow).join(product_hod)\
#.join(product_w2v)

/home/zyc/anaconda2/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [21]:
user_all = user_v.join(user_level).join(user_order)

In [28]:
product_user.to_pickle('product_user.pkl')
user_all.to_pickle('user_all.pickle')
product_all.to_pickle('product_all.pickle')

In [3]:
product_user = pd.read_pickle('product_user.pkl')
pu_trend = pd.read_pickle('pu_trend.pkl')
user_all = pd.read_pickle('user_all.pickle')
product_all = pd.read_pickle('product_all.pickle')

In [4]:
data = product_user.join(pu_trend).reset_index().set_index('user_id').join(user_all)\
.reset_index().set_index('product_id').join(product_all).reset_index()\
.rename(columns={('user_id', ''): 'user_id'})
data['pu_avg_bought'] = data[(u'pureordered', u'pucount')].div(data[('u_order_id', 'u_nunique')])
data['pu_last_bought_gap'] = data[('u_order_id', 'u_nunique')] - data[('puorder_number', 'pumax')]
data['pu_avg_bought_since_first'] = data[('pureordered', 'pucount')].div(data[('u_order_id', 'u_nunique')] - data[('puorder_number', 'pumin')] )

/home/zyc/anaconda2/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)
/home/zyc/anaconda2/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [10]:
"""
label = orders[orders.eval_set == 'train'].set_index('order_id')\
.join(order_products_train.set_index('order_id'))[['product_id', 'user_id']].set_index(['product_id', 'user_id'])
label['label'] = 1
label.to_pickle('label.pkl')
"""
label = pd.read_pickle('label.pkl')

In [11]:
train = orders[orders.eval_set == 'train'].set_index('user_id')\
.join(data.reset_index().set_index('user_id'))\
.reset_index().set_index(['product_id', 'user_id']).join(label)\
.reset_index().fillna(0)

In [13]:
#train.to_pickle('../data/train.pkl')
train.to_csv('../data/train.csv', float_format='%.4f')

KeyboardInterrupt: 

In [27]:
orders[orders.eval_set == 'test'].set_index('user_id')\
.join(data.reset_index().set_index('user_id'))\
.reset_index().fillna(0)\
.to_csv('../data/test.data')